In [2]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

model = AzureChatOpenAI(model="gpt-4o",
                        api_version="2025-04-01-preview",
                        temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요. 제 이름은 안민재입니다."),
    AIMessage("안녕하세요. 안민재님. 반가워요."),
    HumanMessage("제 이름이 뭐에요?")
]

ai_message = model.invoke(messages)

print(ai_message.content)

당신의 이름은 안민재입니다! 😊


In [3]:
# Streaming

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕!"),
]
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 😊 어떻게 도와드릴까요?

In [6]:
# PromptTemplate
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
                                      다음 요리의 레시피를 생각해주세요.
                                      요리명: {dish}
                                      """)

prompt_value = prompt.invoke({"dish": "카레"})
print(prompt_value.text)


                                      다음 요리의 레시피를 생각해주세요.
                                      요리명: 카레
                                      


In [ ]:
# ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from pprint import pprint
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
        ("ai", "{recipe}"), #("assistant", "") 도 가능
    ]
)
prompt_value = prompt.invoke({"dish": "카레", "recipe": "카레 레시피"})
pprint(prompt_value.messages)

[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='카레', additional_kwargs={}, response_metadata={}),
 AIMessage(content='카레 레시피', additional_kwargs={}, response_metadata={})]


In [20]:
# Messages Placeholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요, 저는 안민재입니다.!"),
            AIMessage(content="안녕하세요, 안민재님. 반가워요."),
        ],
        "input": "제 이름이 뭐에요?",
    }
)

pprint(prompt_value.messages)

[SystemMessage(content='You are helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕하세요, 저는 안민재입니다.!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 안민재님. 반가워요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='제 이름이 뭐에요?', additional_kwargs={}, response_metadata={})]


In [21]:
ai_message = model.invoke(prompt_value)
print(ai_message.content)

당신의 이름은 안민재입니다! 😊


In [ ]:
# Output Parsers
# PydanticOutputParser

from pydantic import BaseModel, Field

class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")



from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
